<a href="https://colab.research.google.com/github/awadell1/MLOnAGooseberryBush/blob/master/src/LSTM_noclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import text_to_word_sequence
from tqdm.notebook import tqdm
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
SEED = 42 #setting the seed for reproducebility
AUTOTUNE = tf.data.experimental.AUTOTUNE #currently not entirely sure what this can do

In [ ]:
save_path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors100.tsv'
word2vec_map = pd.read_csv(save_path,sep='\t',header=None)
print(word2vec_map.shape)
word2vec_map.head(2)

(33167, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.264070,-0.292869,-0.120290,-0.502183,0.409219,0.133390,0.418358,-0.319643,-0.169770,0.375747,0.001047,-0.036405,-0.053634,-0.367648,0.247227,0.131370,0.090884,0.079203,-0.197034,-0.338718,-0.231927,0.065685,-0.097877,0.298307,0.186365,0.398437,-0.017262,0.253889,-0.070448,0.329990,0.415608,-0.260716,0.177142,-0.323851,-0.171481,-0.247889,0.209458,-0.245567,0.093872,-0.236581,...,0.457694,0.093809,0.162528,0.254220,-0.155697,-0.016840,0.316370,0.072141,-0.333458,0.366920,0.091554,0.233144,-0.113931,0.226183,0.287019,-0.303512,-0.023759,-0.269189,-0.068603,0.047875,0.374010,-0.006584,0.000182,0.089257,0.215925,-0.251165,0.034845,-0.107870,0.225868,-0.099101,-0.194125,-0.274009,-0.028488,-0.128960,0.238589,0.203563,0.383318,-0.325063,0.279072,-0.001845
1,-0.367691,-0.180021,-0.581465,0.102577,0.115282,0.055618,0.031822,0.019066,-0.296952,0.043763,-0.330150,0.517496,0.027572,-0.191822,-0.002043,-0.198733,-0.257538,0.954689,-0.127911,-0.230018,0.308664,-0.418525,0.208620,0.225576,-0.061917,0.315588,-0.528157,-0.178618,-0.141423,-0.461726,0.381662,-0.537500,-0.280325,0.042296,0.289965,0.318626,0.365973,0.192489,0.368424,-0.203051,...,-0.382657,0.230836,0.051053,0.335296,0.249298,-0.250909,-0.468625,0.229336,-0.460099,-0.077277,-0.006552,0.204857,-0.342267,0.099275,0.003094,0.046386,0.186515,-0.226988,-0.254718,0.287885,-0.247938,0.062137,-0.193653,-0.287656,-0.677540,0.182576,-0.294254,-0.304651,0.113725,0.122848,0.316588,-0.335598,0.012292,0.238053,-0.072420,-0.097892,0.029341,-0.166146,0.371941,-0.345186


In [ ]:
raw_df=pd.read_csv('/content/drive/MyDrive/24787 Group Project  /Data/winemag-data-130k-v2.csv',index_col=0)
raw_data=raw_df[['description']]
raw_label=raw_df[['points']]

In [ ]:
def review_cleanup(x):
  review_ls=text_to_word_sequence(x[0],filters='­­"…–â€˜‘’—!#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r“”')
  return review_ls

In [ ]:
tqdm.pandas()
raw_data['review_separated']=raw_data.progress_apply(lambda x: review_cleanup(x),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
I2W_df=pd.read_csv('/content/drive/My Drive/24787 Group Project  /Data/for_final_submission/Dictionary_new.csv')

I2W=I2W_df.to_dict(orient='dict')['Vocab']
I2W[0] = ''
W2I = {v: k for k, v in I2W.items()}

In [ ]:
def sentence_to_indices(data,w2ind):
    sentences_num=[]
    for row in tqdm(data):
        temp_mem=[w2ind[word] for word in row]
        sentences_num.append(temp_mem)
    return sentences_num

In [ ]:
sentences_num = sentence_to_indices(data=raw_data['review_separated'],w2ind=W2I)

In [ ]:
for seq in sentences_num[:2]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[15, 873, 202, 13, 1978, 3581, 1, 88, 105, 2, 16, 908, 1030, 928, 308, 14444, 48, 60, 1, 88, 421, 142, 360, 18] 
 => 
 ['aromas', 'include', 'tropical', 'fruit', 'broom', 'brimstone', 'and', 'dried', 'herb', 'the', 'palate', "isn't", 'overly', 'expressive', 'offering', 'unripened', 'apple', 'citrus', 'and', 'dried', 'sage', 'alongside', 'brisk', 'acidity']
[6, 7, 27, 1, 75, 3, 8, 17, 7, 99, 61, 123, 144, 78, 22, 28, 1064, 90, 5, 72, 30, 43, 49, 1, 3751, 5, 18, 24, 379, 701, 291, 12, 87, 684, 96, 439, 20, 340] 
 => 
 ['this', 'is', 'ripe', 'and', 'fruity', 'a', 'wine', 'that', 'is', 'smooth', 'while', 'still', 'structured', 'firm', 'tannins', 'are', 'filled', 'out', 'with', 'juicy', 'red', 'berry', 'fruits', 'and', 'freshened', 'with', 'acidity', "it's", 'already', 'drinkable', 'although', 'it', 'will', 'certainly', 'be', 'better', 'from', '2016']


In [ ]:
reviews_num = sentences_num
max_review_length = max([len(seq) for seq in reviews_num])
input_sequences = np.array(pad_sequences(reviews_num, maxlen=max_review_length, padding='pre'))

In [ ]:
for seq in input_sequences[:1]:
  print(f"{seq} \n => \n {[I2W[i] for i in seq]}")

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0    15   873   202    13  1978  3581     1    88
   105     2    16   908  1030   928   308 14444    48    60     1    88
   421   142   360    18] 
 => 
 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
def pretrained_embedding_layer(word_to_vec_map, index_to_word):
    """
    Creates a Keras Embedding() layer
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(index_to_word)+1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map.shape[1]      # define dimensionality of your word vectors (= 256)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for index, word in tqdm(index_to_word.items()): #skip the first item "<padding>"
        emb_matrix[index,:]=word_to_vec_map.iloc[index-1]
      # word_check = dict_pre.get(word)
      # if word_check is not None:
      #     emb_matrix[index,:] = word_to_vec_map[word].to_numpy().reshape(emb_dim,)
      # else:
          
      #     emb_matrix[index,:] = average_average_calculator(index, word_to_vec_map,sequence,dict_pre)

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim,trainable=True,mask_zero=True) 
    ##TODO:is this maskz_zero correct here?
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [ ]:
I2W_nopad=I2W.copy()
embedding_layer = pretrained_embedding_layer(word2vec_map, I2W_nopad)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = -0.5021831


In [ ]:
def WineReviewMachine_V2(input_shape, word_to_vec_map, index_to_word, num_class):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(num_class)(X)
    # Add a softmax activation
    X = Activation("softmax")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
num_class=3
model = WineReviewMachine_V2((max_review_length,), word2vec_map, I2W, num_class)
model.summary()


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 136, 100)          3316900   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 136, 512)          731136    
_________________________________________________________________
dropout_12 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)              

In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(dataset_train,epochs=30,verbose=1,validation_data=dataset_test)

Epoch 1/30
117/117 [==============================] - 64s 546ms/step - loss: 0.5926 - accuracy: 0.7365 - val_loss: 0.5114 - val_accuracy: 0.7739
Epoch 2/30
117/117 [==============================] - 61s 521ms/step - loss: 0.4892 - accuracy: 0.7828 - val_loss: 0.4820 - val_accuracy: 0.7854
Epoch 3/30
117/117 [==============================] - 61s 519ms/step - loss: 0.4568 - accuracy: 0.7995 - val_loss: 0.4699 - val_accuracy: 0.7952
Epoch 4/30
117/117 [==============================] - 61s 524ms/step - loss: 0.4325 - accuracy: 0.8110 - val_loss: 0.4622 - val_accuracy: 0.7963
Epoch 5/30
117/117 [==============================] - 61s 519ms/step - loss: 0.4102 - accuracy: 0.8213 - val_loss: 0.4636 - val_accuracy: 0.7952
Epoch 6/30
117/117 [==============================] - 61s 518ms/step - loss: 0.3883 - accuracy: 0.8323 - val_loss: 0.4730 - val_accuracy: 0.7952
Epoch 7/30
117/117 [==============================] - 61s 518ms/step - loss: 0.3679 - accuracy: 0.8433 - val_loss: 0.4723 - val_ac

In [ ]:
def WineReviewMachine_V3(input_shape, word_to_vec_map, index_to_word):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape = input_shape, dtype = 'int32') #the max length 
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, index_to_word)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    # max_batch_size=1024
    # partitions = tf.range(max_batch_size)
    # num_partitions = max_batch_size
    # x_unpacked = tf.dynamic_partition(embeddings, partitions, num_partitions, name='dynamic_unstack')
    # #x_unpacked = tf.unstack(embeddings) # defaults to axis 0, returns a list of tensors
    # mini_batched_processed=[]
    # for x in x_unpacked:
    #    partitions = tf.range(136)
    #    num_partitions = 136
    #    t_ls = tf.dynamic_partition(x, partitions, num_partitions, name='dynamic_unstack') 
    #    average_embedding = tf.reduce_mean(x, 0)
    #    processed=[]
    #    for t in t_ls:
    #       if tf.math.count_nonzero(t).numpy()[0]==0:
    #          results_tensor=average_embedding
    #       else:
    #          results_tensor=t
    #       processed.append(result_tensor)
    #    batched_embeddings=tf.concat(processed,0)
    #    mini_batched_embeddings.append(batched_embeddings)
    # embeddings=tf.concat(mini_batched_embeddings,0)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=True))(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = Bidirectional(LSTM(256, return_sequences=False))(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(20)(X)
    X = Dense(1)(X)
    # Add a softmax activation
    X = Activation("linear")(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [ ]:
model_linear = WineReviewMachine_V3((max_review_length,), word2vec_map, I2W)
model_linear.summary()


Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 136, 100)          3316900   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 136, 512)          731136    
_________________________________________________________________
dropout_14 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_15 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 20)             

In [ ]:
model_linear.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
#trained 7 epochs
model_linear.fit(dataset_train,epochs=1,verbose=1,validation_data=dataset_test)

117/117 [==============================] - 61s 522ms/step - loss: 0.2224 - mean_squared_error: 0.2224 - val_loss: 0.2222 - val_mean_squared_error: 0.2222


start 300D

In [ ]:
save_path='/content/drive/MyDrive/24787 Group Project  /Data/for_final_submission/vectors300.tsv'
word2vec_map = pd.read_csv(save_path,sep='\t',header=None)
print(word2vec_map.shape)
word2vec_map.head(2)

(33167, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.072741,0.177086,0.046955,-0.161159,-0.262453,-0.022582,-0.175011,0.193300,0.081327,0.513394,0.486482,0.270007,-0.055469,-0.017736,0.233057,-0.339189,0.080459,-0.211314,0.230841,-0.092777,-0.354089,0.353036,-0.032889,-0.039103,-0.100094,-0.262138,-0.219273,-0.020130,-0.142205,0.016919,-0.149281,-0.289330,-0.298905,-0.299549,-0.046952,0.050852,-0.162308,-0.301594,-0.210345,-0.224959,...,0.269495,0.561228,0.110157,0.371961,-0.091705,-0.042529,-0.270073,-0.233123,0.027484,-0.057847,0.025248,0.312116,0.155597,0.338420,0.188724,0.064901,0.057733,0.408761,0.200816,-0.221229,-0.000132,0.308902,-0.411560,0.268989,-0.036705,0.528598,-0.220828,0.026004,-0.039186,-0.078684,-0.030462,0.147154,-0.264258,0.445950,-0.252144,-0.463734,-0.086174,0.195977,0.044055,-0.141106
1,-0.162451,0.056447,0.074096,0.247125,0.275788,-0.075669,-0.405454,0.067573,-0.296444,-0.120678,0.676957,-0.044911,0.046235,0.292248,0.051215,0.366189,-0.120822,0.290646,-0.205931,-0.146317,0.129239,0.328307,-0.261547,0.371859,0.095685,-0.247538,-0.391735,0.014415,0.118940,-0.245449,0.211864,0.195619,0.209814,0.063449,-0.125104,-0.066620,0.020668,-0.013998,-0.467575,0.248410,...,0.273146,0.058361,-0.292375,-0.193279,-0.226322,0.058354,-0.225004,-0.123047,0.010058,-0.657200,-0.042546,0.359351,-0.180970,0.175821,0.042868,0.461206,-0.026787,-0.333164,0.025929,0.368363,-0.224638,-0.160631,0.348195,0.179474,0.058651,0.425275,-0.291038,0.428397,-0.703248,0.350351,0.058166,-0.252873,-0.376265,0.005129,-0.359104,0.110904,0.242587,-0.066842,0.333056,0.044874


In [ ]:
I2W_nopad=I2W.copy()
embedding_layer300 = pretrained_embedding_layer(word2vec_map, I2W_nopad)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])


weights[0][1][3] = -0.5021831


In [ ]:
y_label=raw_label.values
y_label_3levels = []
for i,y0 in enumerate(y_label):
  temp = 0
  if y0 <87:
    temp = int(0)
  elif y0>=87 and y0<93:
     temp = int(1.0)
  elif y0>=93: temp = int(2.0)
  y_label_3levels.append(temp)

In [ ]:
indices = y_label_3levels
depth = max(indices)+1
y_label_one_hot=tf.one_hot(indices, depth)

In [ ]:
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_label_one_hot[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_label_one_hot[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
#this is some tensorflow data cleaning function
dataset_train = tf.data.Dataset.from_tensor_slices((x_train_vec, y_train_label))
dataset_train = dataset_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset_test = tf.data.Dataset.from_tensor_slices((x_test_vec, y_test_label))
dataset_test = dataset_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset_test = dataset_test.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_train)
print(dataset_test)

<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<BatchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
dataset_train = dataset_train.cache().prefetch(buffer_size=AUTOTUNE)
dataset_test = dataset_test.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset_train)
print(dataset_test)

<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>
<PrefetchDataset shapes: ((1024, 136), (1024, 3)), types: (tf.int32, tf.float32)>


In [ ]:
num_class=3
model300 = WineReviewMachine_V2((max_review_length,), word2vec_map, I2W_nopad, num_class)
model300.summary()


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 136)]             0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 136, 300)          9950700   
_________________________________________________________________
bidirectional_16 (Bidirectio (None, 136, 512)          1140736   
_________________________________________________________________
dropout_16 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_17 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)              

In [ ]:
model300.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model300.fit(dataset_train,epochs=30,verbose=1,validation_data=dataset_test)

Epoch 1/30
117/117 [==============================] - 74s 629ms/step - loss: 0.5708 - accuracy: 0.7470 - val_loss: 0.4881 - val_accuracy: 0.7845
Epoch 2/30
117/117 [==============================] - 70s 602ms/step - loss: 0.4638 - accuracy: 0.7949 - val_loss: 0.5032 - val_accuracy: 0.7822
Epoch 3/30
117/117 [==============================] - 71s 605ms/step - loss: 0.4257 - accuracy: 0.8130 - val_loss: 0.5090 - val_accuracy: 0.7846
Epoch 4/30
117/117 [==============================] - 71s 603ms/step - loss: 0.3885 - accuracy: 0.8332 - val_loss: 0.5030 - val_accuracy: 0.7881
Epoch 5/30
117/117 [==============================] - 70s 600ms/step - loss: 0.3496 - accuracy: 0.8529 - val_loss: 0.5238 - val_accuracy: 0.7893
Epoch 6/30
117/117 [==============================] - 70s 601ms/step - loss: 0.3301 - accuracy: 0.8618 - val_loss: 0.5387 - val_accuracy: 0.7877
Epoch 7/30
117/117 [==============================] - 71s 605ms/step - loss: 0.2975 - accuracy: 0.8782 - val_loss: 0.5744 - val_ac

In [ ]:
y_test=raw_label.values-80
#because we have 2 inputs, targets and contexts, I dont know how to use train_test_split on this 
#I created a home grown train_test_spliter

#first, shuffle the indices
targ_ind = np.arange(input_sequences.shape[0])
np.random.shuffle(targ_ind)
#second, assign around 80% of the indices to train

train_ind = targ_ind[:-10000]
test_ind = targ_ind[-10000:]
#third, assign the general data to train and test datatset 
x_train_vec=[]
y_train_label=[]
for i in tqdm(train_ind):
   x_train_vec_temp = input_sequences[i]
   y_train_label_temp = y_test[i]
   x_train_vec.append(x_train_vec_temp)
   y_train_label.append(y_train_label_temp)
x_test_vec=[]
y_test_label=[]
for i in tqdm(test_ind):
   x_test_vec_temp = input_sequences[i]
   y_test_label_temp = y_test[i]
   x_test_vec.append(x_test_vec_temp)
   y_test_label.append(y_test_label_temp)

In [ ]:
model_linear300 = WineReviewMachine_V3((max_review_length,), word2vec_map, I2W_nopad)
model_linear300.summary()


Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 136)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 136, 300)          9950700   
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 136, 512)          1140736   
_________________________________________________________________
dropout_18 (Dropout)         (None, 136, 512)          0         
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 512)               1574912   
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 20)             

In [ ]:
model_linear300.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer='adam', metrics=[tf.keras.losses.MeanSquaredError()])
model_linear300.fit(dataset_train,epochs=30,verbose=1,validation_data=dataset_test)

Epoch 1/30
117/117 [==============================] - 73s 627ms/step - loss: 0.2471 - mean_squared_error: 0.2471 - val_loss: 0.2225 - val_mean_squared_error: 0.2225
Epoch 2/30
117/117 [==============================] - 71s 605ms/step - loss: 0.2243 - mean_squared_error: 0.2243 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 3/30
117/117 [==============================] - 70s 597ms/step - loss: 0.2234 - mean_squared_error: 0.2234 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 4/30
117/117 [==============================] - 70s 599ms/step - loss: 0.2230 - mean_squared_error: 0.2230 - val_loss: 0.2223 - val_mean_squared_error: 0.2223
Epoch 5/30
117/117 [==============================] - 70s 595ms/step - loss: 0.2227 - mean_squared_error: 0.2227 - val_loss: 0.2222 - val_mean_squared_error: 0.2222
Epoch 6/30
117/117 [==============================] - 70s 596ms/step - loss: 0.2226 - mean_squared_error: 0.2226 - val_loss: 0.2222 - val_mean_squared_error: 0.2222
Epoch 7/30

KeyboardInterrupt: ignored